<a href="https://colab.research.google.com/github/suresha97/MSc_Project/blob/main/CNN_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Workspace

In [12]:
#load packages
import scipy
from scipy import signal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
from IPython import display
import copy 
import pickle

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn import preprocessing
from sklearn.model_selection import StratifiedShuffleSplit

import torch
import torchvision
from torchvision import transforms
from torchvision.transforms import Normalize, Resize, ToTensor
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import PIL
from PIL import Image
import cv2

# Helper Functions

In [16]:
#Function to extract windowed segments of spectrograms
def overlap_windows(data, overlap_rate, window_size):
    '''Returns windowed segments of spectrogram

    Args:
        data : spectrogram to segemnts
        window_size : length of each segment
        overlap_rate : subsequent segments have overlap_rates of (1-overlap_rate)%

    '''
    window_list = []
    start = 0
    end = window_size
    remain_length = len(data)
    
    while remain_length>=window_size:
        window_list.append(data[:,int(round(start+0.01)):int(round(end+0.01))])
        start += overlap_rate*window_size #start and end overlap 1
        end += overlap_rate*window_size
        remain_length -= overlap_rate*window_size 
    
    return window_list

def make_data(olr, window_size, augment, fusion):
  '''Loads spectrograms and performace augmentation and or preparation for specifc training metods

  Args:
      olr : specify overlap rate for augment = True
      window_size : specify length of each segment if augment = True
      augment : True for spectrogram augmentation
      fusion : None for single signal or FUSE 1, 'two_channel' for FUSE 2, 'alpha_blend' for FUSE 3#

  '''

  #List of strings for each particiapnt
  parts = []
  nums = [str(x) for x in range(1,10,1)]
  ten = ['0'+x for x in nums]
  others = [str(x) for x in range(10,24,1)]
  participants = ten+others

  ppg_full_list = [] # 32 elements of 40 x 80 x 80 arrays
  resp_full_list = []

  #Load spectorgrams of PPG and RESP signals for each particiapnt
  for part in participants: 
    l_ppg = np.load('/content/drive/My Drive/PHYSIO/Generated_Specs/PPG_120/ppg_spec_{}_120.npy'.format(part)) #40 x 80 x 80
    l_resp = np.load('/content/drive/My Drive/PHYSIO/Generated_Specs/20RESP_120/20resp_spec_{}_120.npy'.format(part)) #40 x 80 x 80 
    
    ppg_full_list.append(l_ppg)
    resp_full_list.append(l_resp)

  ppg_full_array = np.vstack(ppg_full_list)
  resp_full_array = np.vstack(resp_full_list)

  print("PPG:",ppg_full_array.shape)
  print("RESP:",resp_full_array.shape)

  if augment == True:
    #Define resize dimensions for extracted segments
    composed1 = transforms.Compose([ Resize(size=(120,120)),
                                    ToTensor()])

    composed2 = transforms.Compose([ Resize(size=(28,28)),
                                    ToTensor()])
    ppg_aug = []
    resp_aug = []

    #For each sample
    for i in range(len(ppg_full_array)):

      #Extract windowed segemnts for current spectrograms for each signal 
      test_win = overlap_windows(ppg_full_array[i], olr, window_size) 
      test_win2 = overlap_windows(resp_full_array[i], olr, window_size) 
      spec_list = []
      spec_list2 = []

      #Resise current PPG spectrogram segments to 120 x 120 then 28 x 28
      for spec in test_win: 
        out2 = composed1(Image.fromarray(spec))
        out2 = np.transpose(out2.data.numpy(), (1, 2, 0))
        out2 = out2[:,:,0]
        out2 = composed2(Image.fromarray(out2))
        out2 = np.transpose(out2.data.numpy(), (1, 2, 0))
        out2 = out2[:,:,0]
        spec_list.append(out2)

      #Resise current RESP spectrogram segments to 120 x 120 then 28 x 28
      for spec2 in test_win2: 
        out2 = composed1(Image.fromarray(spec2))
        out2 = np.transpose(out2.data.numpy(), (1, 2, 0))
        out2 = out2[:,:,0]
        out2 = composed2(Image.fromarray(out2))
        out2 = np.transpose(out2.data.numpy(), (1, 2, 0))
        out2 = out2[:,:,0]
        spec_list2.append(out2)
      
      ppg_aug.append(np.asarray(spec_list))
      resp_aug.append(np.asarray(spec_list2))

    ppg_aug_array = np.vstack(ppg_aug)
    resp_aug_array = np.vstack(resp_aug)
    print("PPG Augmented :",ppg_aug_array.shape)
    print("RESP Augmented :",resp_aug_array.shape)
    ppg_full_array = ppg_aug_array
    resp_full_array = resp_aug_array

  if fusion == 'two_channel':
    to_stack = [ppg_full_array, resp_full_array]
    two_sig = np.stack(np.asarray(to_stack), axis = 1)
    print("Multi Channel Data:",two_sig.shape)

    return two_sig

  if fusion == 'alpha_blend':
    blend = np.load('/content/drive/My Drive/PHYSIO/spec_alpha_blend.npy')

    if augment == True:
      blend_array = []
      for i in range(len(blend)):

        #Extract windowed segemnts for current spectrograms for each signal 
        test_win = overlap_windows(blend[i], overlap_rate, window_size) 
        spec_list = []

        #Resise current PPG spectrogram segments to 120 x 120 then 28 x 28
        for spec in test_win: 
          out2 = composed1(Image.fromarray(spec))
          out2 = np.transpose(out2.data.numpy(), (1, 2, 0))
          out2 = out2[:,:,0]
          out2 = composed2(Image.fromarray(out2))
          out2 = np.transpose(out2.data.numpy(), (1, 2, 0))
          out2 = out2[:,:,0]
          spec_list.append(out2)

        blend_array.append(np.asarray(spec_list))

      blend_aug = np.vstack(blend_array)
      blend = blend_aug

    print("Alpha-blended Data :",blend.shape)
    return blend

  return ppg_full_array, resp_full_array

def make_labels(file_name,rating,aug_num):
  '''Load lables for training and perform augmentation if necessary

  Args:
      file_name : all_labels for K-means, all_labels_threshold_5.0 for manual thresholding
      rating : string specifying rating  i.e 'val', 'aro', 'val3' ,'aro3', 'class_4' for 4-class K-means
      aug_num : total number of spectrograms after augmentation

  '''
  labels_tdf = pd.read_csv("/content/drive/My Drive/PHYSIO/Generated_Specs/{}.csv".format(file_name)) 
  data_Y = labels_tdf[rating].to_numpy()[0:920]

  if rating == 'class_4':
    #Label encoding for clusterins with quadrant names as labels
    le = preprocessing.LabelEncoder()
    data_Y = le.fit_transform(data_Y)

  #Augment labels to match spectrograms
  aug_labs = []
  for ele in data_Y:
    t_list = [ele]*int(aug_num/920)
    aug_labs.extend(t_list)

  aug_labs = np.asarray(aug_labs)
  print("Labels:",aug_labs.shape)

  return aug_labs

#Accuracy function
def accuracy(features1,features2, labels,task, fuse):
  '''Forward pass through model and return accuracy

  Args:
      features1 : featueres to forward pass 
      features2 : only relevant for FUSE 1
      labels : ground truth classes 
      task : 'binary' for binary classification, 'multi' for all others
      fuse : 1 if FUSE 1 else None
      
  '''
  #Get predictions from model

  if fuse == 1:
    outputs = net.forward(features1, features2)
  else:
    outputs = net.forward(features1)

  if task == 'multi':
    outputs = F.softmax(outputs, dim = 0)
    _, preds = torch.max(outputs.data,1)

  if task == 'binary':
    preds = []
    for el in outputs.data:
      if el >= 0.5:
        preds.append(1)
      elif el < 0.5:
        preds.append(0)

  preds = np.array(preds)

  #Find number that are coorect and accuracy
  num_correct = (torch.tensor(preds).int() == labels.int()).sum().numpy()
  accuracy = num_correct/(labels.size()[0])

  return accuracy*100, preds

# CNN Architecures

## Baseline CNN

In [13]:


class Single_Net(nn.Module):
    
    # Define the neural network layers
    def __init__(self,fcsize,in_channel,num_c1, num_c2,outsize, task):
        
        super(Single_Net, self).__init__()
        self.fcsize = fcsize
        self.task = task
        self.in_channel = in_channel
        self.num_c1 = num_c1
        self.num_c2 = num_c2
        self.outsize = outsize

        #Conv layer
        self.conv1 = nn.Conv2d(self.in_channel, self.num_c1, kernel_size=5)
        self.conv2 = nn.Conv2d(self.num_c1, self.num_c2, kernel_size=5)

        #FC layers
        self.fc1 = nn.Linear(self.fcsize, self.outsize)
        self.fc2 = nn.Linear(self.fcsize*2,1)


    # Define the forward pass.    
    def forward(self, x):
        
        #Conv1 Block 1 - Convolution + Pooling + Non-linearity
        x = F.max_pool2d(self.conv1(x), 2, stride = 2)
        x = F.relu(x)

        #Conv1 Block 2 - Convolution + Pooling + Non-linearity
        x = F.max_pool2d(self.conv2(x), 2, stride = 2)
        x = F.relu(x)

        #Flatten array
        x = x.view(-1, self.fcsize)

        if self.task == 'binary':
          x = torch.sigmoid(self.fc1(x))
        if self.task == 'multi':
          x = self.fc1(x)

        return x


## Baseline CNN for FUSE 1 

In [14]:
'''For Fusion Method 1 (FUSE 1 )'''
class Multi_Net(nn.Module):
    
    # Define the neural network layers
    def __init__(self,fcsize,in_channel,num_c1, num_c2,outsize,task):
        
        super(Multi_Net, self).__init__()
        self.fcsize = fcsize
        self.task = task
        self.in_channel = in_channel
        self.num_c1 = num_c1
        self.num_c2 = num_c2
        self.outsize = outsize

        #Conv layer
        self.conv1 = nn.Conv2d(self.in_channel, self.num_c1, kernel_size=5)
        self.conv2 = nn.Conv2d(self.num_c1, self.num_c2, kernel_size=5)

        #FC layers
        self.fc1 = nn.Linear(self.fcsize, self.outsize)
        self.fc2 = nn.Linear(self.fcsize*2,self.outsize)

    # Define the forward pass.    
    
    def forward(self, x, x2):
    
        #Conv1 Block 1 (Singal 1) - Convolution + Pooling + Non-linearity
        x = F.max_pool2d(self.conv1(x), 2, stride = 2)
        x = F.relu(x)
        
        #Conv1 Block 2 (Singal 1) - Convolution + Pooling + Non-linearity
        x = F.max_pool2d(self.conv2(x), 2, stride = 2)
        x = F.relu(x)

        #Conv1 Block 1 (Singal 2) - Convolution + Pooling + Non-linearity
        x2 = F.max_pool2d(self.conv1(x2), 2)
        x2 = F.relu(x2)
        
        #Conv1 Block 2 (Singal 2) - Convolution + Pooling + Non-linearity
        x2 = F.max_pool2d(self.conv2(x2), 2)
        x2 = F.relu(x2)
        
        #Flatten layers for both signal
        x = x.view(-1, self.fcsize)
        x2 = x2.view(-1, self.fcsize)

        #Concatenate features extracted from both spectrograms
        x_combined = torch.cat((x,x2),dim=1)

        #FC layers, non-linearities and predcition layers
        if self.task == 'binary':
          x_combined = torch.sigmoid(self.fc2(x_combined))

        if self.task == 'multi':
          x_combined = self.fc2(x_combined)
        
        return x_combined

## Attention Model 1 (Att 1) 

In [ ]:
np.random.seed(3)

#Module is a base class for all neural network modules
class Attention_Net2(nn.Module):
    
    # Define the neural network layers
    def __init__(self,fcsize,in_channel,num_c1, num_c2,outsize,task):
        
        super(Single_Net, self).__init__()
        self.fcsize = fcsize
        self.task = task
        self.in_channel = in_channel
        self.num_c1 = num_c1
        self.num_c2 = num_c2
        self.outsize = outsize

        #Conv layers
        self.conv1 = nn.Conv2d(self.in_channel, self.num_c1, kernel_size=5)
        self.conv2 = nn.Conv2d(self._num_c1, self.num_c2, kernel_size=5)
        self.fc1 = nn.Linear(self.fcsize, self.outsize)
  
        self.project = nn.Conv2d(self.num_c1, self.fcsize, kernel_size = 1) 
        self.comp = nn.Conv2d(self.fcsize, 1, kernel_size = 1) 


    # Define the forward pass.    
    def forward(self, x):
        #Conv1 Block 1 - Convolution + Pooling + Non-linearity
        a_l = self.conv1(x)

        x = F.max_pool2d(a_l, 2, stride = 2)
        x = torch.sigmoid(x)

        x = F.max_pool2d(self.conv2(x), 2, stride = 2)
        x = torch.sigmoid(x)
        
        #Save global output of CNN to another variable
        N,_,_,_ = x.size()
        g = x.view(N,288,1,1)

        #Project features onto same dimesnion as global output and add the two
        p_l = self.project(a_l)  
        N,C,W,H = p_l.size()
        comb = p_l+g

        #Get Compatability scores and Normalise to get attention map
        c = self.comp(comb)
        a = F.softmax(c.view(N,1,-1), dim =2).view(N,1,W,H)
        
        #Sum feature vectors extracted after conv1, wieghted with attention map
        g_a = torch.mul(a.expand_as(p_l),p_l)
        g_a = g_a.view(N,C,-1).sum(dim=2)

        if self.task == 'binary':
          g_a = torch.sigmoid(self.fc1(g_a))

        if self.task == 'multi':
          g_a = self.fc1(g_a)
        
        return g_a
        

## Attention Model 2 (Att 2)

In [17]:
np.random.seed(3)

#Module is a base class for all neural network modules
class Attention_Net2(nn.Module):
    
    # Define the neural network layers
    def __init__(self,fcsize,in_channel,num_c1, num_c2,outsize,task):
        
        super(Attention_Net2, self).__init__()
        self.fcsize = fcsize
        self.task = task
        self.in_channel = in_channel
        self.num_c1 = num_c1
        self.num_c2 = num_c2
        self.outsize = outsize

        #Conv layers
        self.conv1 = nn.Conv2d(self.in_channel, self.num_c1, kernel_size=5)
        self.conv2 = nn.Conv2d(self.num_c1, self.num_c2, kernel_size=5)
        self.fc1 = nn.Linear(self.num_c2, 1)

        self.tanlayer = nn.Conv2d(self.num_c2,self.num_c2, kernel_size = 1)
        self.comp = nn.Conv2d(self.num_c2, 1, kernel_size = 1) 
      
    # Define the forward pass.    
    def forward(self, x):
        #Conv1 Block 1 - Convolution + Pooling + Non-linearity
        a_l = self.conv1(x)

        x = F.max_pool2d(a_l, 2, stride = 2)
        x = F.relu(x)

        x = F.max_pool2d(self.conv2(x), 2, stride = 2)
        x = F.relu(x)
        
        #Save global output of CNN to another variable
        N,C,W,H = x.size()
        A = x

        #New representation of CNN output vectors
        # sending each vector throguh MLP by passing through conv layer with kernel size 1 and same number of output channels.
        new_A = F.tanh(self.tanlayer(A)) 
        new_A = new_A*0.3 #lambda

        #Get normalised compatiability scores
        E = self.comp(new_A)
        alpha = F.softmax(E.view(N,1,-1), dim =2).view(N,1,W,H)

        #Sum feature vectors extracted after conv1, wieghted with attention map
        alpha = torch.mul(alpha.expand_as(A),A)
        c = alpha.view(N,C,-1).sum(dim=2)

        if self.task == 'binary':
          c = torch.sigmoid(self.fc1(c))

        if self.task == 'multi':
          c = self.fc1(c)

        
        return c
        

# Training Function

In [18]:
# Train the CNN
def train_net(train_set, no_epochs, lr, m, opt, task, fuse):

    #Define tloss functionhe loss
    if task == 'multi':
      loss_func = nn.CrossEntropyLoss()
    if task == 'binary':
      loss_func = nn.BCELoss()

    #Define Optimiser
    if opt == 'ADAM':
      optimizer = optim.AdamW(net.parameters(), lr = lr)
    elif opt == 'SGD':
      optimizer=optim.SGD(net.parameters(), lr = lr, momentum = m, nesterov = True)

    best_val_loss = 10000
    best_epoch = 0
    best_val_acc = 0

    losses_train = []
    losses_val = []

    # Loop over the number of epochs
    for epoch in range(no_epochs):

        #Initialise loss and acc
        current_loss = 0.0
        current_accuracy = 0.0

        # Loop over each mini-batch
        for batch_index, training_batch in enumerate(train_set, 0):
  
            #Load the mini-batch and wrap with variable
            if fuse == 1:
              inputs, inputs2, labels = training_batch
              inputs, inputs2, labels = inputs, inputs2, labels
            else:
              inputs, labels = training_batch
              inputs,  labels = inputs, labels

            #Initalise parameter gradients
            optimizer.zero_grad()

            #Forward pass
 
            if fuse == 1:
              outputs = net.forward(inputs, inputs2)
            else:
              outputs = net.forward(inputs)

            if task == 'multi':
              labels = labels.long()

            loss = loss_func(outputs, labels)
            
            #Backward pass
            loss.backward()
            optimizer.step()

            #Add loss 
            current_loss += loss.item()

            #Add accuracy to the overall accuracy

            if fuse == 1:
              current_accuracy += accuracy(inputs,inputs2,labels, task, fuse)[0]
            else:
              current_accuracy += accuracy(inputs,'_',labels, task, fuse)[0]


        print('[Epoch: %d Batch: %5d] loss: %.3f, acc: %.3f' % (epoch + 1, batch_index+1, current_loss/batchsize, current_accuracy/batchsize))
        losses_train.append(current_loss/batchsize)

    print("------------------------------------------------------")
    print('Training has finished')


    if fuse == 1:
      test_ac, preds = accuracy(testf_tensor, testf_tensor2, testl_tensor, task, fuse)
    else:
      test_ac, preds = accuracy(testf_tensor, '_', testl_tensor, task, fuse)

    print('Test Accuracy: ',test_ac)

    #Test F1 score
    true_y = testl_tensor.detach().numpy()
    F1_test = sklearn.metrics.f1_score(true_y, preds, average = 'weighted')
    print('Test F1 Score :', F1_test)
    print("All :",sklearn.metrics.classification_report(true_y, preds))
    print("Confusion Matrix :")
    print(sklearn.metrics.confusion_matrix(true_y, preds))
    #sklearn.metrics.plot_confusion_matrix(best_model, testf_tensor)
    
    #Plot learning curves
    fig = plt.figure(figsize=(5,5), dpi= 80, facecolor='w', edgecolor='k')
    length = np.arange(0,len(losses_train),1)
    plt.plot(length,losses_train,'r',linewidth = 2)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

    return test_ac, F1_test, sklearn.metrics.confusion_matrix(true_y, preds)

# Load and make Training Data

In [19]:
#Choose spectrogram files

#Choose data augmentation parameters for running augment = True
window_size = 40
overlap_rate = 1

print("----------- Single Input -----------")
#Choose clustering strategy and rating to load 
file_n = 'all_labels_threshold_5.0' #all_labels for k-means, all_labels_threshold_5.0 for manual threhsold, class_4 only in all_labels
ratings = 'val' #val/aro for binary classification, val3/aro3 for multiclass, class_4 for 4 class classification
num_participants = 23

#Split data by particiapnts for LOSO CV *ALSO NEED FILEN_NAME FOR SPECTROGRAM FILES*
ppg_array, resp_array = make_data(overlap_rate, window_size, augment=True, fusion = None)
sub_splits_ppg = np.split(ppg_array, num_participants)
sub_splits_resp = np.split(resp_array, num_participants)

print()
print("----------- FUSE 2 -----------")

'''For Fusion Method 2 (FUSE 2)'''
two_s = make_data(overlap_rate, window_size, augment=True, fusion ='two_channel')
sub_splits_both = np.split(two_s, 23)

print()
print("----------- FUSE 3 -----------")
'''For Fusion Method 3 (FUSE 3)'''
alpha_s = make_data(overlap_rate, window_size, augment=True, fusion = 'alpha_blend')
sub_splits_alpha = np.split(alpha_s, 23)

print()
print("----------- Labels -----------")
len_aug = len(ppg_array)
labs = make_labels(file_n, ratings, len_aug)
sub_splits_labs = np.split(labs, 23)

print()
print("----------- LOSO -----------")
print("LOSO Single Signal Features:",np.asarray(sub_splits_resp).shape)
print("LOSO Multi-channel Features :",np.asarray(sub_splits_both).shape)
print("LOSO Alpha-blend Features:",np.asarray(sub_splits_alpha).shape)
print("LOSO Labels:",np.asarray(sub_splits_labs).shape)


#Choose input method (PPG, RESP, FUSE 2, FUSE 3)
dat = sub_splits_ppg
#dat2 = sub_splits_resp

----------- Single Input -----------
PPG: (920, 120, 120)
RESP: (920, 120, 120)
PPG Augmented : (2760, 28, 28)
RESP Augmented : (2760, 28, 28)

----------- FUSE 2 -----------
PPG: (920, 120, 120)
RESP: (920, 120, 120)
PPG Augmented : (2760, 28, 28)
RESP Augmented : (2760, 28, 28)
Multi Channel Data: (2760, 2, 28, 28)

----------- FUSE 3 -----------
PPG: (920, 120, 120)
RESP: (920, 120, 120)
PPG Augmented : (2760, 28, 28)
RESP Augmented : (2760, 28, 28)
Alpha-blended Data : (2760, 28, 28)

----------- Labels -----------
Labels: (2760,)

----------- LOSO -----------
LOSO Single Signal Features: (23, 120, 28, 28)
LOSO Multi-channel Features : (23, 120, 2, 28, 28)
LOSO Alpha-blend Features: (23, 120, 28, 28)
LOSO Labels: (23, 120)


# Train Model using LOSO CV

In [ ]:
image_size = np.asarray(dat).shape[2]
subjects = [x for x in range(0,23,1)]

accuracies_test = []
f1s = []
cf_list = []
#Train model 23 times - differnet particiapnt data each time 
for s in subjects:
  start_time = time.time()

  train_inds = np.where(np.asarray(subjects) != s)[0]
  train_feats = np.take(np.asarray(dat), train_inds, axis = 0)
  train_feats = np.vstack(train_feats)
  
  train_labs = np.take(np.asarray(sub_splits_labs), train_inds, axis = 0)
  train_labs = np.reshape(train_labs, (train_feats.shape[0]))
  print("Training Features",train_feats.shape)
  print("Training Labels",train_labs.shape)

  #Get test fold
  test_feats = np.take(np.asarray(dat), s, axis = 0)  
  test_labs = np.take(np.asarray(sub_splits_labs), s, axis = 0)
  test_labs = np.reshape(test_labs, (test_feats.shape[0]))
  print("Training Features",test_feats.shape)
  print("Training Labels",test_labs.shape)

  #Reshape train and test arrays to specify number of input chanenels
  train_feats = train_feats.reshape(len(train_feats),1,image_size,image_size)
  test_feats = test_feats.reshape(len(test_feats),1,image_size,image_size)

  #Convert to torch tensors
  trainf_tensor = torch.tensor(train_feats, dtype=torch.float)
  trainl_tensor = torch.tensor(train_labs, dtype=torch.float)
  testf_tensor = torch.tensor(test_feats, requires_grad=False, dtype=torch.float)
 
  testl_tensor = torch.tensor(test_labs, requires_grad=False, dtype=torch.float)
  print("Trainf",trainf_tensor.shape)
  print("Trainl",trainl_tensor.shape)
  print("Testf",testf_tensor.shape)
  print("Testl",testl_tensor.shape)

  ''' For Fusion Method 1 (FUSE 1)'''
  # train_feats2 = np.take(np.asarray(dat2), train_inds, axis = 0)
  # train_feats2 = np.vstack(train_feats2)
  # test_feats2 = np.take(np.asarray(dat2), s, axis = 0)
  # train_feats2 = train_feats.reshape(len(train_feats2),1,image_size,image_size)
  # test_feats2 = test_feats.reshape(len(test_feats2),1,image_size,image_size)
  # trainf_tensor2 = torch.tensor(train_feats2, dtype=torch.float)
  # testf_tensor2 = torch.tensor(test_feats2, requires_grad=False, dtype=torch.float)
  # print("Trainf",trainf_tensor2.shape)

  #Batchify training data using trainloader
  batch_size = 25
  trainset = torch.utils.data.TensorDataset(trainf_tensor, trainl_tensor)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
  batchsize = len(trainloader)

  print("Participant : ",s+1)

  #Change num_c1 to 12 and num_c2 to 24 for FUSE 1/3. Change in_channel to 2 for and fuse to 2 for FUSE 1. fcsize 288 or 384 or 480]
  #net = Single_Net(fcsize = 480, in_channel = 1, num_c1 = 15, num_c2 = 30, outsize = 4, task = 'multi' )
  #net = Multi_Net(fcsize = 384, in_channel = 1, num_c1 = 12, num_c2 = 24, outsize = 3, task = 'multi' )
  #net = Attention_Net1(fcsize = 288, in_channel = 1, num_c1 = 9, num_c2 = 18, outsize = 1, task = 'binary' )
  net = Attention_Net2(fcsize = 288, in_channel = 1, num_c1 = 9, num_c2 = 18, outsize = 1, task = 'binary' )
  test_accuracy,testf1,cf = train_net(trainloader, no_epochs = 10, lr = 0.001, m = 0.0, opt = 'ADAM', task = 'binary', fuse = None)    
  accuracies_test.append(test_accuracy) 
  f1s.append(testf1)
  cf_list.append(cf)
  print("Pre-processing time  validation sets --- %s minutes ---" % (((time.time() - start_time)/60)))

#Print average metrics across all participants
avg_acc = sum(accuracies_test)/len(accuracies_test)
print("Accuracies :", accuracies_test)
print("Average test accuracy: {}%".format(round(avg_acc,2)))
print("Average test F1 Score :", sum(f1s)/len(f1s))